# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary.

### Data Task Definition

The goal is to build a supervised regression model to find the relationship between the price of a used car and its features. The target variable is the price of the car, and the features include various attributes such as make, model, year, mileage, and condition. The goal is to identify which features are most influential in determining the price of a used car and to provide actionable insights for the used car dealership to optimize their inventory and pricing strategies.

This task involves data preprocessing, feature engineering, and multiple model building using regression techniques. The models' coefficients will be compared to identify the most significant drivers of used car prices. The performance of the models will be evaluated using appropriate metrics such as Mean Absolute Error (MAE) or Root Mean Squared Error (RMSE) to ensure that the predictions are accurate and reliable for business decision making.


### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

#### Steps for Data Understanding
1. Load the dataset and perform an initial inspection to understand its structure, the number of rows and columns, data types, and a preview the first and last records.
2. Check for missing values and determine how to handle them, fill them with mean, mode, or delete them.
3. Explore the distribution of the target variable (price) and identify any outliers.
4. Identify any categorical variables and explore unique values.
5. Identify any data quality issues, such as duplicates, inconsistency, or errors, and plan how to address them.

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [23]:
df = pd.read_csv('data/vehicles.csv')

df.shape

(426880, 18)

***Interpretation:*** The dataset contains 426,000 rows and 20 columns. This indicates that we have a large dataset to work with, which can provide valuable insights into the factors that influence used car prices.

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [25]:

null_total = df.isnull().sum().sort_values(ascending=False)

null_percentage = (df.isnull().sum() * 100/ df.isnull().count() ).sort_values(ascending=False)
null_df = pd.DataFrame({'Total Nulls': null_total, 'Percentage of Nulls': null_percentage})
null_df

,Total Nulls,Percentage of Nulls
size,306361,71.767476
cylinders,177678,41.622470
condition,174104,40.785232
VIN,161042,37.725356
drive,130567,30.586347
paint_color,130203,30.501078
type,92858,21.752717
manufacturer,17646,4.133714
title_status,8242,1.930753
model,5277,1.236179


***Interpretation:*** The above dataframe shows the total number of null values and their percentage for each column in the dataset. The dataframe is sorted by the total number of nulls in descending order, the variable with the highest number of nulls will appear first.

The `size` column has the highest number of null values, missing `71.8%` of its entries, which indicates that it may not be a reliable feature for modeling. The `cylinder` column is missing `41.6%` of its entries, which is also a significant amount of missing data. The `condition` column is missing `40.79%` of entries, also a significant amount. The `drive` and `paint_color` columns are missing `30.5%` of netries. The `type` column is missin `21.8%` of entries. The other columns are missing less than `5%` of their entries, which may be more manageable.

I will handle the missing columns as follows:
- For `size`, I will drop this column from the dataset due to the high percentage of missing values.
- For `cylinder`, I will consider fill the missing values with the most (mode) common value.
- For `condition`, I will also consider filling the missing values with the most common value, as it is a categorical variable.
- For `drive`, `paint_color`, and `type`, I will also consider filling the missing values with the most common value, as they are categorical variables and this approach can help preserve the overall distribution of the data.
- For other columns with less than `5%` missing values, I will consider filling the missing values with the mean (for numerical variables) or the mode (for categorical variables), because the missing values is relatively low.

In [26]:
df.describe()

,id,price,year,odometer
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07


In [27]:
df.head(20)

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc
5,7222379453,hudson valley,1600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ny
6,7221952215,hudson valley,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ny
7,7220195662,hudson valley,15995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ny
8,7209064557,medford-ashland,5000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,or
9,7219485069,erie,3000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pa


In [28]:
df.tail()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
426875,7301591192,wyoming,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,1N4AA6AV6KC367801,fwd,NaN,sedan,NaN,wy
426876,7301591187,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,7JR102FKXLG042696,fwd,NaN,sedan,red,wy
426877,7301591147,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,1GYFZFR46LF088296,NaN,NaN,hatchback,white,wy
426878,7301591140,wyoming,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,58ABK1GG4JU103853,fwd,NaN,sedan,silver,wy
426879,7301591129,wyoming,30590,2019.0,bmw,4 series 430i gran coupe,good,NaN,gas,22716.0,clean,other,WBA4J1C58KBM14708,rwd,NaN,coupe,NaN,wy


In [29]:
# Check for unique values in categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    print(f"{col}: {df[col].nunique()} unique values")

region: 404 unique values
manufacturer: 42 unique values
model: 29649 unique values
condition: 6 unique values
cylinders: 8 unique values
fuel: 5 unique values
title_status: 6 unique values
transmission: 3 unique values
VIN: 118246 unique values
drive: 3 unique values
size: 4 unique values
type: 13 unique values
paint_color: 12 unique values
state: 51 unique values


In [30]:
# Why does the model column have so many unique values?  Print unique values to understand what is going on
for col in categorical_cols:
    print(f"{col}: {df[col].unique()[:10]}")  # Print first 10 unique values for each categorical column



region: ['prescott' 'fayetteville' 'florida keys' 'worcester / central MA'
 'greensboro' 'hudson valley' 'medford-ashland' 'erie' 'el paso'
 'bellingham']
manufacturer: [nan 'gmc' 'chevrolet' 'toyota' 'ford' 'jeep' 'nissan' 'ram' 'mazda'
 'cadillac']
model: [nan 'sierra 1500 crew cab slt' 'silverado 1500' 'silverado 1500 crew'
 'tundra double cab sr' 'f-150 xlt' 'sierra 2500 hd extended cab'
 'silverado 1500 double' 'tacoma' 'colorado extended cab']
condition: [nan 'good' 'excellent' 'fair' 'like new' 'new' 'salvage']
cylinders: [nan '8 cylinders' '6 cylinders' '4 cylinders' '5 cylinders' 'other'
 '3 cylinders' '10 cylinders' '12 cylinders']
fuel: [nan 'gas' 'other' 'diesel' 'hybrid' 'electric']
title_status: [nan 'clean' 'rebuilt' 'lien' 'salvage' 'missing' 'parts only']
transmission: [nan 'other' 'automatic' 'manual']
VIN: [nan '3GTP1VEC4EG551563' '1GCSCSE06AZ123805' '3GCPWCED5LG130317'
 '5TFRM5F17HX120972' '1GT220CG8CZ231238' '1GCVKREH6GZ228691'
 '1GCHTCE37G1186784' '1G1YR3DW3B51021

***Interpretation:*** The `model` is a string that contains a specific car model (e.g. 'sierra 1500 crew cab slt' or 'silverado 1500'), which is why it has so many unique values. This is not a categorical variable that we can easily use in modeling without any form of encoding or transformation. I need to consider how to handle this variable in our modeling process.

In [40]:
counts, bin_edges = np.histogram(df['price'], bins=50)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

agg_df = pd.DataFrame({"bin_center": bin_centers, "count": counts})
fig = px.bar(agg_df, x="bin_center", y="count")
fig.update_layout(title='Distribution of Car Prices')
fig.show()

***Interpretation:*** The distribution is not clearly seen because there are some extreme outliers in the price variable. To better visualize the distribution, I will apply a logarithmic transformation to the price variable and then plot the histogram again.

In [49]:
prices = np.log1p(df["price"]).to_numpy()
counts, bin_edges = np.histogram(prices, bins=50)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

agg_df = pd.DataFrame({"price": bin_centers, "count": counts})
fig = px.bar(agg_df, x="price", y="count")
fig.update_layout(title_text='Distribution of Car Prices (Log Transformed)')
fig.show()

***Interpretation***: The price in this data has some extreme outliers. The log transformation helps to reduce the impact of these outliers and provides a more normal distribution of the price variable. However, I will remove the outliers using the IQR method.

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`.

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.